In [116]:
import urllib.request
import shutil
import geopandas as gpd
import pandas as pd
from bs4 import BeautifulSoup
from typing import List, Dict

## Download file

In [118]:
def download_zip(url: str, save_dir: str) -> None:
    # init opener and add headers
    opener = urllib.request.build_opener()
    opener.addheaders = [('User-agent', 'Mozilla/5.0')]
    urllib.request.install_opener(opener)
    
    # retrieve and download file to repository
    try:
        urllib.request.urlretrieve(url, save_dir)
        print(f"Downloaded temp.zip from {url}")
    except Exception as e:
        raise(e)
        
    return 

In [129]:
def unzip_file(zipfile_dir: str, target_dir: str = "../data/") -> None:
    shutil.unpack_archive(zipfile_dir, target_dir, "zip")
    print(f"Unpacked files into {target_dir}")
    return

In [130]:
url = "https://data.gov.sg/dataset/aeaf4704-5be1-4b33-993d-c70d8dcc943e/download"
save_dir = "../data/raw.zip"
download_zip(url=url, save_dir=save_dir)
unzip_file(zipfile_dir=save_dir)

Downloaded temp.zip from https://data.gov.sg/dataset/aeaf4704-5be1-4b33-993d-c70d8dcc943e/download
Unpacked files into ../data/


##  Extract Data

In [140]:
raw_df = gpd.read_file("../data/hawker-centres-geojson.geojson")

In [141]:
raw_df.head()

,Name,Description,geometry
0,kml_1,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.93873 1.33199 0.00000)
1,kml_2,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.81834 1.28733 0.00000)
2,kml_3,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.82899 1.37238 0.00000)
3,kml_4,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.86674 1.36316 0.00000)
4,kml_5,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.83703 1.35201 0.00000)


### Parsing lat long

In [156]:
lat_lon_df = pd.DataFrame()

In [157]:
lat_lon_df["LONGITUDE"] = raw_df["geometry"].x
lat_lon_df["LATITUDE"] = raw_df["geometry"].y

In [158]:
lat_lon_df.head()

,LONGITUDE,LATITUDE
0,103.938733,1.331987
1,103.818339,1.287331
2,103.828994,1.372385
3,103.866737,1.363157
4,103.837032,1.352007


### Parsing description

In [133]:
# Function to parse html description
def parse_description(description_list: List[List[str]]) -> List[Dict[str, str]]:
    result = []
    for des in description_list:
        soup = BeautifulSoup(des)
        table = soup.find('table')
        header, values = table.find_all('th')[1:], table.find_all('td')
        dict_values = {header[i].text: values[i].text for i in range(len(header)) if values[i].text != ""}
        result.append(dict_values)
    return result

In [134]:
description_dict = parse_description(raw_df["Description"].values.tolist())
description_df = pd.DataFrame.from_dict(description_dict)
description_df.head(n=2)

,ADDRESSBLOCKHOUSENUMBER,EST_ORIGINAL_COMPLETION_DATE,STATUS,LANDYADDRESSPOINT,PHOTOURL,DESCRIPTION,NAME,ADDRESSTYPE,HUP_COMPLETION_DATE,LANDXADDRESSPOINT,...,INC_CRC,FMEL_UPD_D,APPROXIMATE_GFA,AWARDED_DATE,ADDRESSBUILDINGNAME,IMPLEMENTATION_DATE,CLEANINGSTARTDATE,INFO_ON_CO_LOCATORS,CLEANINGENDDATE,RNR_STATUS
0,85,30/6/1977,Existing,34910.13,http://www.nea.gov.sg/images/default-source/Ha...,HUP Standard Upgrading,Bedok North Street 4 Blk 85 (85 Fengshan Centre),I,20/12/2012,39731.49,...,CFC780A1B5DC7721,20210330151704,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,85,4/4/1972,Existing,29972.02,http://www.nea.gov.sg/images/default-source/Ha...,HUP Reconfiguration,Redhill Lane Blk 85 (Redhill Food Centre),I,17/6/2005,26332.89,...,1D515CA502CE0A60,20210330151704,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [135]:
# checking data quality
description_df.isnull().sum()

ADDRESSBLOCKHOUSENUMBER           8
EST_ORIGINAL_COMPLETION_DATE      1
STATUS                            0
LANDYADDRESSPOINT                 0
PHOTOURL                         11
DESCRIPTION                       0
NAME                              0
ADDRESSTYPE                      21
HUP_COMPLETION_DATE              28
LANDXADDRESSPOINT                 0
ADDRESSSTREETNAME                 2
ADDRESSPOSTALCODE                 7
ADDRESS_MYENV                    11
INC_CRC                           0
FMEL_UPD_D                        0
APPROXIMATE_GFA                  92
AWARDED_DATE                    108
ADDRESSBUILDINGNAME             110
IMPLEMENTATION_DATE             108
CLEANINGSTARTDATE               114
INFO_ON_CO_LOCATORS             111
CLEANINGENDDATE                 114
RNR_STATUS                      114
dtype: int64

### Overall dataframe

In [160]:
df = pd.concat([lat_lon_df, description_df], axis=1)

In [164]:
df.head(n=2)

,LONGITUDE,LATITUDE,ADDRESSBLOCKHOUSENUMBER,EST_ORIGINAL_COMPLETION_DATE,STATUS,LANDYADDRESSPOINT,PHOTOURL,DESCRIPTION,NAME,ADDRESSTYPE,...,INC_CRC,FMEL_UPD_D,APPROXIMATE_GFA,AWARDED_DATE,ADDRESSBUILDINGNAME,IMPLEMENTATION_DATE,CLEANINGSTARTDATE,INFO_ON_CO_LOCATORS,CLEANINGENDDATE,RNR_STATUS
0,103.938733,1.331987,85,30/6/1977,Existing,34910.13,http://www.nea.gov.sg/images/default-source/Ha...,HUP Standard Upgrading,Bedok North Street 4 Blk 85 (85 Fengshan Centre),I,...,CFC780A1B5DC7721,20210330151704,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,103.818339,1.287331,85,4/4/1972,Existing,29972.02,http://www.nea.gov.sg/images/default-source/Ha...,HUP Reconfiguration,Redhill Lane Blk 85 (Redhill Food Centre),I,...,1D515CA502CE0A60,20210330151704,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [163]:
df.to_csv("../data/collated_hawkercentre_data.csv")